In [1]:
# Importing libraries
import pandas as pd
import numpy as np
import keras
import tensorflow
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Embedding, Flatten, Dense

In [2]:
# Importing dataset
data = pd.read_csv("train.txt", sep=';')
data.columns = ["Text", "Emotions"]
data.head()

,Text,Emotions
0,i can go from feeling so hopeless to so damned...,sadness
1,im grabbing a minute to post i feel greedy wrong,anger
2,i am ever feeling nostalgic about the fireplac...,love
3,i am feeling grouchy,anger
4,ive been feeling a little burdened lately wasn...,sadness


In [3]:
# Start by tokenizing the data:
texts = data["Text"].tolist()
labels = data["Emotions"].tolist()

# Tokenize the text data
tokenizer = Tokenizer()
tokenizer.fit_on_texts(texts)

In [4]:
# Padding the sequences of the texts to have the same length:
sequences = tokenizer.texts_to_sequences(texts)
max_length = max([len(seq) for seq in sequences])
padded_sequences = pad_sequences(sequences, maxlen=max_length)

In [5]:
# Encode the string labels to integers
label_encoder = LabelEncoder()
labels = label_encoder.fit_transform(labels)

In [6]:
# One-hot encode the labels
one_hot_labels = keras.utils.to_categorical(labels)

In [7]:
# Split the data into training and testing sets
xtrain, xtest, ytrain, ytest = train_test_split(padded_sequences, one_hot_labels, test_size=0.2, random_state = 0)

In [8]:
# creating the model
model = Sequential()
model.add(Embedding(input_dim=len(tokenizer.word_index) + 1, 
                    output_dim=128, input_length=max_length))
model.add(Flatten())
model.add(Dense(units=128, activation="relu"))
model.add(Dense(units=len(one_hot_labels[0]), activation="softmax"))

model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
model.fit(xtrain, ytrain, epochs=10, batch_size=32, validation_data=(xtest, ytest))



Epoch 1/10


400/400 [==============================] - 27s 63ms/step - loss: 1.3435 - accuracy: 0.4854 - val_loss: 0.8381 - val_accuracy: 0.7188
Epoch 2/10
400/400 [==============================] - 27s 67ms/step - loss: 0.3308 - accuracy: 0.8997 - val_loss: 0.5631 - val_accuracy: 0.8191
Epoch 3/10
400/400 [==============================] - 27s 68ms/step - loss: 0.0595 - accuracy: 0.9856 - val_loss: 0.5882 - val_accuracy: 0.8263
Epoch 4/10
400/400 [==============================] - 28s 69ms/step - loss: 0.0246 - accuracy: 0.9945 - val_loss: 0.6753 - val_accuracy: 0.8184
Epoch 5/10
400/400 [==============================] - 29s 74ms/step - loss: 0.0165 - accuracy: 0.9970 - val_loss: 0.6914 - val_accuracy: 0.8178
Epoch 6/10
400/400 [==============================] - 28s 71ms/step - loss: 0.0135 - accuracy: 0.9973 - val_loss: 0.6955 - val_accuracy: 0.8194
Epoch 7/10
400/400 [==============================] - 26s 64ms/step - loss: 0.0120 - accuracy: 0.9975 - val_loss: 0.7142 - val_accur

In [11]:
# Taking a sentence as an input text and see how the model performs:
input_text = "She didn't come today because she lost her dog yestertay!"

# Preprocess the input text
input_sequence = tokenizer.texts_to_sequences([input_text])
padded_input_sequence = pad_sequences(input_sequence, maxlen=max_length)
prediction = model.predict(padded_input_sequence)
predicted_label = label_encoder.inverse_transform([np.argmax(prediction[0])])
print(predicted_label)

1/1 [==============================] - 0s 46ms/step
['sadness']
